In [1]:
import os
import cv2
import glob 
import joblib
import itertools
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsClassifier
os.chdir("..")
os.getcwd()

'/home/devkumar.patel/Desktop/ScleraWorkflow/Datasets/AI-Powered-Framework-for-Sclera-Diagnosis-Assistance/Code'

In [2]:
from Image.Processor import Utils

/home/devkumar.patel/miniconda3/envs/Rare/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
paths = {
            "Jaundice": (glob.glob("dataset/Scleras/Jaundice/*.png"),0, 'train'),
            "Jaundice-Test": (glob.glob("dataset/Scleras/Jaundice/test/*.png"),0, 'test'),
            "Normal": (glob.glob("dataset/Scleras/Normal/*.png"),1, 'train'),
            "Normal-Test": (glob.glob("dataset/Scleras/Normal/test/*.png"),1, 'test'),
            "OseteogensisImperfecta" : (glob.glob("dataset/Scleras/OsteogensisImperfecta/*.png"),2, 'train'),
            "OseteogensisImperfecta-Test" : (glob.glob("dataset/Scleras/OsteogensisImperfecta/test/*.png"),2, 'test')
        }

# Augment the data: 
paths["Jaundice"] = (list(itertools.chain.from_iterable(itertools.repeat(x, 40) for x in paths["Jaundice"][0])), 0, 'train')
paths["OseteogensisImperfecta"] = (list(itertools.chain.from_iterable(itertools.repeat(x, 40 ) for x in paths["OseteogensisImperfecta"][0])), 2, 'train')


train = []
train_hist = []
train_label = []
test_label = []
test_hist = []
test = []

for dataset_names in paths.keys():
    print( dataset_names ,": ",len(paths[dataset_names][0])  )
    dataset, label, mode = paths[dataset_names]
    for i, name in enumerate(dataset): 
        if mode == 'train':
            train_hist.append(Utils.Image_To_Histogram(cv2.imread(name), concat=True ).flatten()  )
            train.append(name)
            train_label.append(label)
        if mode == 'test':
            test_hist.append(Utils.Image_To_Histogram(cv2.imread(name), concat=True ).flatten()  )
            test.append(name)
            test_label.append(label)

print(f"Total Train: {len(train)}")
print(f"Total Test: {len(test)}")


Jaundice :  240
Jaundice-Test :  1
Normal :  435
Normal-Test :  65
OseteogensisImperfecta :  360
OseteogensisImperfecta-Test :  2
Total Train: 1035
Total Test: 68


In [4]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_hist, train_label)

KNeighborsClassifier()

In [5]:
accuracy = knn.score(test_hist, test_label)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.99


In [9]:
def predict_img(img_path, label, model):
    print(f"Path: {img_path}")
    print(f"label: {label}")
    
    img = cv2.imread(img_path)
    hist = Utils.Image_To_Histogram(  img,  concat=True  ).flatten() 
    hist = hist.reshape(1, -1)
    print(f"Predicted: {knn.predict(hist)[0]}")

predict_img(paths["OseteogensisImperfecta-Test"][0][1], paths["OseteogensisImperfecta-Test"][1], knn)
#predict_img(paths["Jaundice-Test"][0][0], paths["Jaundice-Test"][1], knn)
#predict_img("/home/devkumar.patel/Desktop/ScleraWorkflow/Datasets/AI-Powered-Framework-for-Sclera-Diagnosis-Assistance/Code/dataset/Scleras/OsteogensisImperfecta/test/KKHFV250011R.png", paths["OseteogensisImperfecta-Test"][1], knn)



Path: dataset/Scleras/OsteogensisImperfecta/test/25001_copy3R.png
label: 2
Predicted: 1


In [10]:
filename = 'Image/classifiers/knn.joblib'
joblib.dump(knn, filename)

['Image/classifiers/knn.joblib']